# OpenCode as a txtai LLM

[OpenCode](https://github.com/anomalyco/opencode) is an open source AI coding agent. It's rapidly growing in popularity as an open alternative to [Claude Code](https://code.claude.com/docs/en/overview). 

OpenCode shows the power of permissive open source. It's rapidly undergoing mass adoption and as of January 2026 sits at `81K+` GitHub ⭐'s. OpenCode supports running as a [local server](https://opencode.ai/docs/server/) via `opencode serve`.

This notebook will explore how TxtAI integrates this as a LLM pipeline.

# Install dependencies

Install `txtai` and all dependencies.

In [ ]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[api,pipeline-llm]

# Install OpenCode if not already installed and run serve
!curl -fsSL https://opencode.ai/install | bash
!opencode serve &

# Create a OpenCode LLM

Now that everything is installed, let's test it out!

In [6]:
from IPython.display import Markdown, display

from txtai import LLM

def generate(request):
    display(Markdown(llm(request)))

# Connect to OpenCode server and use default LLM provider
llm = LLM("opencode")

generate("Show a Python Hello World example")

```python
print("Hello, World!")
```

OK, now let's try this for a more advanced use case.

In [8]:
generate("Show a simple and basic TxtAI RAG example")

```python
from txtai import Embeddings, LLM

# Create embeddings model
embeddings = Embeddings(path="sentence-transformers/all-MiniLM-L6-v2")

# Create LLM
llm = LLM("huggingface/Qwen/Qwen2.5-1.5B-Instruct")

# Sample documents
documents = [
    "Python is a programming language created by Guido van Rossum",
    "Machine learning is a subset of artificial intelligence",
    "Deep learning uses neural networks with multiple layers"
]

# Build the index
embeddings.index([(i, doc) for i, doc in enumerate(documents)])

# RAG function
def rag_query(question):
    # Search for relevant documents
    results = embeddings.search(question, 2)
    
    # Build context from retrieved documents
    context = " ".join([documents[result[0]] for result in results])
    
    # Generate response using LLM with context
    prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
    return llm(prompt)

# Example usage
question = "What is Python?"
answer = rag_query(question)
print(f"Question: {question}")
print(f"Answer: {answer}")
```

Interesting! It's not limited to Python though.

In [9]:
generate("Show a x86 assembly hello world")

```assembly
section .data
    hello db 'Hello, World!', 10    ; String with newline
    hello_len equ $ - hello         ; Length of string

section .text
    global _start

_start:
    ; sys_write system call
    mov eax, 4                      ; sys_write
    mov ebx, 1                      ; stdout
    mov ecx, hello                  ; message
    mov edx, hello_len              ; message length
    int 0x80                        ; kernel interrupt

    ; sys_exit system call
    mov eax, 1                      ; sys_exit
    mov ebx, 0                      ; exit code 0
    int 0x80                        ; kernel interrupt
```

Compile and run:
```bash
nasm -f elf32 hello.asm -o hello.o
ld -m elf_i386 hello.o -o hello
./hello
```

In [11]:
generate("""
Fix the following error in the code below and explain what's wrong.

Error:
AttributeError: 'NoneType' object has no attribute 'startswith'

Code:
a = None
a.startswith("hello")
""")

```python
a = None
if a is not None:
    a.startswith("hello")
else:
    print("a is None, cannot call startswith")
```

**What's wrong:** The variable `a` is `None`, which is a `NoneType` object. The `startswith()` method only exists for strings, not for `NoneType`. You need to check if `a` is not `None` before calling string methods on it.

# OpenAI-compatible Endpoint

Now that OpenCode is integrated into the TxtAI ecosystem, it opens a lot of different opportunities. Let's say we want to host a local OpenAI compatible endpoint, no problem!

Write the following file.

## config.yml

```yaml
# Enable OpenAI compat endpoint
openai: True

llm:
  path: opencode
```

and start the following process.

```
CONFIG=config.yml uvicorn "txtai.api:app"
```

In [16]:
from IPython.display import Markdown, display

from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="api-key",
)

response = client.chat.completions.create(
    messages=[{
        "role": "user",
        "content": "Show a Python factorial function",
    }],
    model="opencode",
)

display(Markdown((response.choices[0].message.content)))

```python
def factorial(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

# Example usage:
# print(factorial(5))  # Output: 120
```

Just like before we get an answer! This time via the OpenAI client. Fun times.

# Wrapping up

This notebook showed how to connect TxtAI and OpenCode. This will lead to some interesting integrations. One such example is [ncoder](https://github.com/neuml/ncoder), an AI coding agent that integrates with Jupyter Notebooks. Stay tuned for more!